In [1]:
import os
import shutil

# Ruta de la carpeta con las imágenes
carpeta_origen = 'rx'  

# Crear subcarpetas de destino
for i in range(1, 5):
    os.makedirs(os.path.join(carpeta_origen, f'carpeta{i}'), exist_ok=True)

# Filtrar solo imágenes
extensiones_validas = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tif', '.tiff')
imagenes = sorted([f for f in os.listdir(carpeta_origen) if f.lower().endswith(extensiones_validas)])

# Repartir equitativamente en 4 carpetas
for i, imagen in enumerate(imagenes):
    carpeta_destino = os.path.join(carpeta_origen, f'carpeta{(i % 4) + 1}')
    ruta_origen = os.path.join(carpeta_origen, imagen)
    ruta_destino = os.path.join(carpeta_destino, imagen)
    shutil.move(ruta_origen, ruta_destino)

print("Distribución completada.")

Distribución completada.


In [14]:
import os

carpeta = 'img_test/Lote2'  # <-- cámbiala por la tuya

for archivo in os.listdir(carpeta):
    ruta_actual = os.path.join(carpeta, archivo)

    if os.path.isfile(ruta_actual):
        nombre_base, _ = os.path.splitext(archivo)
        nuevo_nombre = nombre_base + '.jpg'
        nueva_ruta = os.path.join(carpeta, nuevo_nombre)
        
        # Si ya existe un archivo con el nuevo nombre, añade un sufijo numérico
        contador = 1
        while os.path.exists(nueva_ruta):
            nuevo_nombre = f"{nombre_base}_{contador}.jpg"
            nueva_ruta = os.path.join(carpeta, nuevo_nombre)
            contador += 1

        os.rename(ruta_actual, nueva_ruta)

print("Renombrado completado evitando duplicados.")


Renombrado completado evitando duplicados.


In [23]:
import os

# Ruta con las imagenes
carpeta = 'rx'
extensiones_validas = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tif', '.tiff')

# Obtener lista de imágenes y ordenarlas
imagenes = sorted([f for f in os.listdir(carpeta) if f.lower().endswith(extensiones_validas)])

# Renombrar cada imagen
for i, nombre in enumerate(imagenes, start=1):
    extension = os.path.splitext(nombre)[1]
    nuevo_nombre = f'rx{i}{extension}'
    ruta_vieja = os.path.join(carpeta, nombre)
    ruta_nueva = os.path.join(carpeta, nuevo_nombre)
    os.rename(ruta_vieja, ruta_nueva)

print("Renombrado completado.")


Renombrado completado.


In [ ]:
# ======================================================================================================
# Experimento 03: Medición de la proyección de los ángulos de los caninos sobre la línea media central
# Se utilizan dos modelos
# 1) Un modelo U-Net para segmentar la forma del canino (máscara)
# 2) Un modelo YoloV5 para detectar la posición de los caninos e incisivos centrales

import cv2
import numpy as np
import torch
import os
import math
import sys
sys.path.append('../src')
from tooth_shape_model_unet import UNet
from torchvision import transforms
from PIL import Image 


# ======================================================================
# Modelo U-Net para segmentación de caninos (obtener la forma del canino, para trazar la línea sobre él)

IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CLASSES = 3  # fondo, canino izquierdo, canino derecho

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
shape_unet = UNet(num_classes=NUM_CLASSES).to(device)
shape_unet.load_state_dict(torch.load('models/tooth_shape_unet.pth', map_location=device))
shape_unet.eval()

infer_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# ======================================================================
# Modelo pre-entrenado (YoloV5) para la detección posicional de caninos e incisivos centrales
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load(
    'ultralytics/yolov5', 
    'custom',
    path='models/tooth_detection.pt',
    force_reload=True
).to(device)

# Umbral de confianza de detección posicional de caninos
CONF_THRESHOLD = 0.85
model.conf = CONF_THRESHOLD
print(f"Umbral de confianza ajustado a: {CONF_THRESHOLD}")

# =================================================================================================
# Bloque principal

# Path de la imagen de entrada
# Sobre ésta imagen se ejecutará la detección de caninos y la segmentación de la forma del canino
img_path = 'Muestra/rx1.jpg'
if not os.path.exists(img_path):
    print(f"ERROR: La imagen no existe en {img_path}")
    exit()

img = cv2.imread(img_path)
if img is None:
    print(f"ERROR: No se pudo leer la imagen desde {img_path}")
    exit()

print(f"Imagen cargada correctamente: {img.shape}")
orig = img.copy()
height, width = img.shape[:2]

results = model(img)
df = results.pandas().xyxy[0]  # xmin, ymin, xmax, ymax, confidence, class, name

print("\n=== RESULTADOS DE DETECCIÓN ===")
print(df)

def calculate_line_intersection(line1_p1, line1_p2, line2_p1, line2_p2):
    """
    Calcula el punto de intersección entre dos líneas definidas por dos puntos cada una.
    Si las líneas son paralelas o no se intersectan dentro de los límites de la imagen, devuelve None.
    
    Args:
        line1_p1, line1_p2: Puntos que definen la primera línea
        line2_p1, line2_p2: Puntos que definen la segunda línea
        
    Returns:
        tuple: Coordenadas (x, y) del punto de intersección o None si no hay intersección
    """
    # Línea 1: (x1, y1) a (x2, y2)
    x1, y1 = line1_p1
    x2, y2 = line1_p2
    
    # Línea 2: (x3, y3) a (x4, y4)
    x3, y3 = line2_p1
    x4, y4 = line2_p2
    
    # Calcular denominador para verificar si las líneas son paralelas
    denom = (y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1)
    
    if denom == 0:  # Líneas paralelas
        return None
    
    # Calcular el punto de intersección
    ua = ((x4 - x3) * (y1 - y3) - (y4 - y3) * (x1 - x3)) / denom
    
    # Punto de intersección
    x = x1 + ua * (x2 - x1)
    y = y1 + ua * (y2 - y1)
    
    return (int(x), int(y))

def extend_line_to_boundaries(point1, point2, img_width, img_height, midline_x=None):
    """
    Extiende una línea definida por dos puntos hasta los límites de la imagen o hasta intersectar con la línea media.
    
    Args:
        point1 (tuple): Coordenadas (x, y) del primer punto.
        point2 (tuple): Coordenadas (x, y) del segundo punto.
        img_width (int): Ancho de la imagen.
        img_height (int): Alto de la imagen.
        midline_x (int, opcional): Coordenada x de la línea media. Si se proporciona, la línea se extenderá hasta esta línea.
        
    Returns:
        tuple: Un par de tuplas con las coordenadas de los puntos extendidos (p1_extended, p2_extended).
    """
    x1, y1 = point1
    x2, y2 = point2
    
    # Si los puntos son iguales, no se puede definir una dirección
    if x1 == x2 and y1 == y2:
        return point1, point2
    
    # Calcular la dirección de la línea
    dx = x2 - x1
    dy = y2 - y1
    
    # Si la línea es vertical
    if dx == 0:
        # Extender hasta los bordes superior e inferior
        return (x1, 0), (x1, img_height)
    
    # Calcular la pendiente y el intercepto
    m = dy / dx
    b = y1 - m * x1
    
    # Puntos extendidos
    extended_points = []
    
    # Si hay una línea media definida, calcular la intersección con ella
    if midline_x is not None:
        # Calcular el punto de intersección con la línea media
        y_intersect = m * midline_x + b
        
        # Verificar si la intersección está dentro de los límites de la imagen
        if 0 <= y_intersect <= img_height:
            # Determinar en qué lado de la línea media está el punto original
            if (x1 < midline_x and x2 < midline_x) or (x1 > midline_x and x2 > midline_x):
                # Ambos puntos están en el mismo lado de la línea media
                # Extender hasta la línea media en una dirección
                if x1 < midline_x:
                    extended_points.append((midline_x, int(y_intersect)))
                else:
                    extended_points.append((midline_x, int(y_intersect)))
            elif (x1 < midline_x and x2 > midline_x) or (x1 > midline_x and x2 < midline_x):
                # No es necesario extender hasta la línea media
                pass
    
    # Intersecciones con los bordes de la imagen
    
    # Intersección con y=0 (borde superior)
    if abs(m) > 0.0001:  # No es una línea horizontal
        x_top = (0 - b) / m
        if 0 <= x_top <= img_width:
            extended_points.append((int(x_top), 0))
    
    # Intersección con y=img_height (borde inferior)
    if abs(m) > 0.0001:  # No es una línea horizontal
        x_bottom = (img_height - b) / m
        if 0 <= x_bottom <= img_width:
            extended_points.append((int(x_bottom), img_height))
    
    # Intersección con x=0 (borde izquierdo)
    y_left = b
    if 0 <= y_left <= img_height:
        extended_points.append((0, int(y_left)))
    
    # Intersección con x=img_width (borde derecho)
    y_right = m * img_width + b
    if 0 <= y_right <= img_height:
        extended_points.append((img_width, int(y_right)))
    
    # Si no hay suficientes puntos de intersección, usar los puntos originales
    if len(extended_points) < 2:
        return point1, point2
    
    # Ordenar los puntos extendidos según su distancia desde el punto medio entre p1 y p2
    midpoint = ((x1 + x2) / 2, (y1 + y2) / 2)
    
    # Si estamos del lado izquierdo de la línea media y queremos extender hacia la línea media
    if midline_x is not None and (x1 < midline_x and x2 < midline_x):
        # Encontrar el punto más cercano al borde y el punto más cercano a la línea media
        points_sorted = sorted(extended_points, key=lambda p: p[0])  # Ordenar por coordenada x
        return points_sorted[0], points_sorted[-1]  # El primero es el más a la izquierda, el último es el más a la derecha
    
    # Si estamos del lado derecho de la línea media y queremos extender hacia la línea media
    elif midline_x is not None and (x1 > midline_x and x2 > midline_x):
        # Encontrar el punto más cercano al borde y el punto más cercano a la línea media
        points_sorted = sorted(extended_points, key=lambda p: p[0], reverse=True)  # Ordenar por coordenada x (reverso)
        return points_sorted[0], points_sorted[-1]  # El primero es el más a la derecha, el último es el más a la izquierda
    
    # En otros casos, simplemente usar las dos intersecciones más alejadas entre sí
    else:
        # Calcular todas las combinaciones de distancias entre puntos
        max_dist = 0
        p1_ext, p2_ext = extended_points[0], extended_points[1]
        
        for i in range(len(extended_points)):
            for j in range(i + 1, len(extended_points)):
                dist = math.sqrt((extended_points[i][0] - extended_points[j][0])**2 + 
                                (extended_points[i][1] - extended_points[j][1])**2)
                if dist > max_dist:
                    max_dist = dist
                    p1_ext, p2_ext = extended_points[i], extended_points[j]
        
        return p1_ext, p2_ext

def get_center(detection):
    xmin, ymin, xmax, ymax = map(int, [detection['xmin'], detection['ymin'], 
                                      detection['xmax'], detection['ymax']])
    cx = (xmin + xmax) // 2
    cy = (ymin + ymax) // 2
    return (cx, cy)

def get_corners(detection, side):
    if side == 'izq':
        return (int(detection['xmax']), int(detection['ymax'])), (int(detection['xmin']), int(detection['ymin']))
    elif side == 'der':
        return (int(detection['xmin']), int(detection['ymax'])), (int(detection['xmax']), int(detection['ymin']))
    else:   
        raise ValueError("Lado no válido. Debe ser 'izq' o 'der'.")


# Detecciones de incisivos y caninos
incisor_detections = df[df['name'] == 'inc']
canines  = df[df['name'] == 'canine']

# Línea media desde incisivos
inc_center = None
if len(incisor_detections) > 0:
    best_inc = incisor_detections.sort_values('confidence', ascending=False).iloc[0]
    inc_center = get_center(best_inc)
    # Dibujar línea media
    cv2.line(orig, (inc_center[0], 0), (inc_center[0], height), (0, 255, 0), 2)
    cv2.putText(orig, 'Línea media', (inc_center[0] + 10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    print(f"Línea media trazada en x={inc_center[0]}")
else:
    print("No se detectaron incisivos; no se puede trazar la línea media.")

import math

def segment_full_and_crop(orig_img, roi_coords):
    # Segmenta toda la imagen y recorta ROI
    img_rgb = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img_rgb)
    x = infer_transform(pil).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = shape_unet(x)
        probs  = torch.softmax(logits, dim=1)[0]  # [3,H,W]

    # si side=='left'
    ch = probs[1] if side=='left' else probs[2]
    heatmap = (ch.cpu().numpy() * 255).astype(np.uint8)
    h, w = orig_img.shape[:2]
    heatmap_full = cv2.resize(heatmap, (w, h), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(f"debug/heat_full_{side}.png", heatmap_full)
    chan = probs[1] if side=='left' else probs[2]
    mask = (chan.cpu().numpy() > 0.03).astype(np.uint8) * 255
    # redimensionar a full-res
    h,w = orig_img.shape[:2]
    mask_full = cv2.resize(mask,(w,h),interpolation=cv2.INTER_NEAREST)
    # recortar
    # x1,y1,x2,y2 = roi_coords
    return mask_full


def calculate_angle(line_p1, line_p2, vertical_line_x):
    """
    Calcula el ángulo entre una línea definida por dos puntos y una línea vertical.
    
    Args:
        line_p1 (tuple): Primer punto de la línea.
        line_p2 (tuple): Segundo punto de la línea.
        vertical_line_x (int): Coordenada x de la línea vertical.
        
    Returns:
        float: Ángulo en grados entre las líneas.
    """
    # Verificar que los puntos no sean iguales
    if line_p1[0] == line_p2[0] and line_p1[1] == line_p2[1]:
        return 0  # No se puede calcular el ángulo si los puntos son iguales
    
    # Vector de la línea
    vector_line = (line_p2[0] - line_p1[0], line_p2[1] - line_p1[1])
    
    # Vector de la línea vertical (0, 1) normalizado
    vector_vertical = (0, 1)
    
    # Calcular el ángulo entre los vectores usando el producto punto
    # Normalizar los vectores
    magnitude_line = math.sqrt(vector_line[0]**2 + vector_line[1]**2)
    
    if magnitude_line == 0:
        return 0
    
    unit_vector_line = (vector_line[0] / magnitude_line, vector_line[1] / magnitude_line)
    
    # Producto punto de los vectores unitarios
    dot_product = unit_vector_line[0] * vector_vertical[0] + unit_vector_line[1] * vector_vertical[1]
    
    # Asegurarse de que el producto punto esté en el rango [-1, 1]
    dot_product = max(-1.0, min(1.0, dot_product))
    
    # Calcular el ángulo en radianes y convertirlo a grados
    angle_rad = math.acos(dot_product)
    angle_deg = math.degrees(angle_rad)
    
    # Determinar la dirección del ángulo (positivo o negativo)
    # Si el punto p2 está a la derecha de la línea vertical, el ángulo es positivo
    # Si está a la izquierda, el ángulo es negativo
    direction = 1 if (line_p1[0] < vertical_line_x and line_p2[0] > vertical_line_x) or \
                    (line_p1[0] > vertical_line_x and line_p2[0] < vertical_line_x and line_p1[1] > line_p2[1]) else -1
    
    # Ajustar el ángulo según el cuadrante
    if unit_vector_line[0] < 0:
        angle_deg = 180 - angle_deg
    
    # Asegurarse de que el ángulo esté entre 0 y 180 grados
    if angle_deg > 90:
        angle_deg = 180 - angle_deg
        
    return angle_deg * direction

def process_and_draw_canine(det, orig_img, side, inc_center_x=None):
    coords = (int(det['xmin']), int(det['ymin']), int(det['xmax']), int(det['ymax']))
    # Extraemos Cordenadas
    x1, y1, x2, y2 = coords
    roi = orig_img[coords[1]:coords[3], coords[0]:coords[2]]
    # 1) Segmentar
    mask_roi = segment_full_and_crop(roi, coords)
    cv2.imwrite(f"debug/mask_roi_{side}.png", mask_roi)
    # 2) Extraer contorno
    cnts,_ = cv2.findContours(mask_roi,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return orig_img
    contour = max(cnts, key=cv2.contourArea).reshape(-1,2).astype(np.float32)
    # 3) PCA
    mean,vecs,_ = cv2.PCACompute2(contour, mean=None)
    axis = vecs[0]
    # 4) Extremos
    dif = contour - mean
    projs = dif.dot(axis.T)
    p1 = list(contour[np.argmin(projs)].astype(int))
    p2 = list(contour[np.argmax(projs)].astype(int))
    if p1[1] > p2[1]: p1, p2 = p2, p1
    
    # Coordenadas punto 1
    p1x = p1[0]
    p1y = p1[1]
    # Coordenadas punto 2
    p2x = p2[0]
    p2y = p2[1]
    
    if (side == "izq") and (p1x > p2x):
        p1[1] = p2y
        p2[1] = p1y
    if (side == "der") and (p1x < p2x):
        p1[1] = p2y
        p2[1] = p1y
        
    # Ajustar a coords global
    p1g = (p1[0]+coords[0], p1[1]+coords[1])
    p2g = (p2[0]+coords[0], p2[1]+coords[1])
    # 5) Dibujar
    cv2.circle(orig_img, p1g, 4,(0,255,0),-1)
    cv2.circle(orig_img, p2g, 4,(0,255,0),-1)

    # Agregar punto (sacar despues)
    cv2.putText(orig_img, 'p1', (p1g[0] + 10, p1g[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(orig_img, 'p2', (p2g[0] + 10, p2g[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Extender la línea hasta los límites de la imagen o hasta la línea media
    h, w = orig_img.shape[:2]
    midline_x = inc_center_x if inc_center_x is not None else None
    
    exp1, exp2 = extend_line_to_boundaries(p1g, p2g, w, h, midline_x)
    
    cv2.line(orig_img, exp1, exp2, (0,0,255), 2)
    
    # Para los angulos
    # Coordenada central en X
    center_x = (x1 + x2) // 2
    # Coordenada para poner el texto justo debajo de la caja
    text_y = y2 + 30

    if midline_x is not None:
        midline_top = (midline_x, 0)
        midline_bottom = (midline_x, h)
        intersection = calculate_line_intersection(exp1, exp2, midline_top, midline_bottom)
        
        if intersection:
            cv2.circle(orig_img, intersection, 6, (255, 0, 255), -1)
            angle = calculate_angle(exp1, exp2, midline_x)
            angle_text = f"Angulo: {abs(angle):.1f} Deg"
            cv2.putText(orig_img, angle_text, 
            (center_x - 50, text_y),  # desplazamos 50 px a la izquierda para centrar mejor el texto
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
            
            radius = 40
            start_angle = 90
            
            if angle < 0:
                end_angle = 90 - abs(angle)
            else:
                end_angle = 90 + abs(angle)
            
            cv2.ellipse(orig_img, intersection, (radius, radius), 
                        0, min(start_angle, end_angle), max(start_angle, end_angle), 
                        (255, 0, 255), 2)
            
            #cv2.putText(orig_img, "Intersección", 
             #          (intersection[0] + 10, intersection[1]), 
              #         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
    
    return orig_img


# Procesamiento de caninos usando el nuevo enfoque de análisis de forma
for index, det in canines.iterrows():
    center = get_center(det)
    
    # Determinar lado según x vs línea media
    side = 'izq' if center[0] < inc_center[0] else 'der'
    
    # Dibujar bounding box del canino
    x1, y1, x2, y2 = map(int, [det['xmin'], det['ymin'], det['xmax'], det['ymax']])
    color = (255, 0, 0) if side == 'izq' else (0, 0, 255)
    cv2.rectangle(orig, (x1, y1), (x2, y2), color, 2)

    # Procesar y dibujar el canino con la línea extendida hasta la línea media
    lined = process_and_draw_canine(det, orig, side, inc_center[0] if inc_center else None)
    #cv2.imwrite('dental_shape_analysis_unet.jpg', lined)
    
    # Mostrar información de la detección
    cv2.putText(orig, f"Canino {side}: {det['confidence']:.2f}", 
                (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Dibujar el centro del diente 
    cv2.circle(orig, center, 5, color, -1)

# Dibujar todas las detecciones con centros
def draw_box(det):
    if det['name'] not in ['inc', 'canine']:
        x1, y1, x2, y2 = map(int, [det['xmin'], det['ymin'], det['xmax'], det['ymax']])
        color = (0, 255, 255)
        cv2.rectangle(orig, (x1, y1), (x2, y2), color, 2)
        cv2.putText(orig, f"{det['name']}: {det['confidence']:.2f}", 
                   (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        c = get_center(det)
        cv2.circle(orig, c, 5, color, -1)

# Solo dibujamos las cajas para otras detecciones que no sean incisivos o caninos
for _, det in df.iterrows():
    if det['name'] not in ['inc', 'canine']:
        draw_box(det)

# Guardar y mostrar la imagen resultante
cv2.imwrite('dental_shape_analysis.jpg', orig)
cv2.destroyAllWindows()
##

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\josem/.cache\torch\hub\master.zip


YOLOv5  2025-6-20 Python-3.12.3 torch-2.7.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
C:\Users\josem/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Umbral de confianza ajustado a: 0.85
Imagen cargada correctamente: (1280, 2440, 3)

=== RESULTADOS DE DETECCIÓN ===
          xmin        ymin         xmax        ymax  confidence  class    name
0  1137.677368  463.714294  1307.943481  780.272339    0.937926      1     inc
1   994.286987  339.954651  1124.865356  613.205200    0.917207      0  canine
2  1336.144165  352.465576  1449.814331  663.742249    0.908585      0  canine
Línea media trazada en x=1222
